In [1]:

from tsl.datasets import MetrLA, PemsBay
import nannyml as nml
import numpy as np

dataset1 = MetrLA()
dataset2 = PemsBay()
df1 = dataset1.dataframe()
df2 = dataset2.dataframe()

# Description of values of MetrLA Dataset

In [2]:
df1.describe([el for el in np.arange(0.05, 1, 0.05)])

nodes,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
channels,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
count,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,...,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000
mean,61.882286,64.501686,64.430367,52.741089,49.397003,53.992199,58.450401,63.319408,60.626286,56.617855,...,40.392975,62.305073,56.006786,63.867298,65.022072,62.638168,66.929962,58.057335,63.816082,54.429813
std,12.123010,3.459473,10.358665,9.591479,15.163940,10.781991,11.355241,3.018191,8.511759,19.331457,...,9.454754,14.384436,16.112831,4.425472,7.496623,3.688378,3.470031,8.977792,9.480382,17.394672
min,2.500000,10.875000,8.111111,1.000000,3.000000,4.857143,5.000000,8.000000,1.750000,1.000000,...,1.000000,3.625000,2.875000,15.875000,5.571429,2.875000,6.000000,2.000000,5.444445,1.428571
5%,21.750000,58.500000,29.750000,30.625000,25.777779,27.875000,30.666666,58.000000,48.222221,10.666667,...,22.555555,18.500000,12.125000,54.777779,53.250000,55.580000,61.000000,40.666668,46.500000,10.222222
10%,56.875000,61.000000,60.888889,42.250000,29.888889,40.375000,35.125000,59.777779,54.253571,15.900000,...,25.500000,45.750000,19.555555,57.750000,59.777779,58.222221,63.625000,43.777779,58.500000,15.250000
15%,60.500000,62.375000,64.000000,48.333332,32.333332,46.136607,49.625000,60.750000,56.000000,27.857143,...,28.065000,61.375000,49.222221,60.125000,62.750000,59.750000,65.125000,46.500000,61.375000,38.250000
20%,62.222221,63.125000,65.375000,50.222221,34.125000,48.625000,56.125000,61.333332,58.000000,44.569446,...,30.111111,64.125000,54.555557,61.666668,64.222221,60.750000,65.888885,50.000000,63.250000,54.125000
25%,63.125000,63.625000,66.000000,51.125000,36.250000,50.750000,58.138392,61.875000,59.250000,59.500000,...,34.125000,65.222221,57.875000,62.666668,65.111115,61.333332,66.444443,53.625000,64.285713,57.625000


# Description of values of PemsBay Dataset

In [3]:
df2.describe([el for el in np.arange(0.05, 1, 0.05)])

nodes,773869,767541,767542,717447,717446,717445,773062,767620,737529,717816,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
channels,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
count,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,...,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000,34272.000000
mean,61.882286,64.501686,64.430367,52.741089,49.397003,53.992199,58.450401,63.319408,60.626286,56.617855,...,40.392975,62.305073,56.006786,63.867298,65.022072,62.638168,66.929962,58.057335,63.816082,54.429813
std,12.123010,3.459473,10.358665,9.591479,15.163940,10.781991,11.355241,3.018191,8.511759,19.331457,...,9.454754,14.384436,16.112831,4.425472,7.496623,3.688378,3.470031,8.977792,9.480382,17.394672
min,2.500000,10.875000,8.111111,1.000000,3.000000,4.857143,5.000000,8.000000,1.750000,1.000000,...,1.000000,3.625000,2.875000,15.875000,5.571429,2.875000,6.000000,2.000000,5.444445,1.428571
5%,21.750000,58.500000,29.750000,30.625000,25.777779,27.875000,30.666666,58.000000,48.222221,10.666667,...,22.555555,18.500000,12.125000,54.777779,53.250000,55.580000,61.000000,40.666668,46.500000,10.222222
10%,56.875000,61.000000,60.888889,42.250000,29.888889,40.375000,35.125000,59.777779,54.253571,15.900000,...,25.500000,45.750000,19.555555,57.750000,59.777779,58.222221,63.625000,43.777779,58.500000,15.250000
15%,60.500000,62.375000,64.000000,48.333332,32.333332,46.136607,49.625000,60.750000,56.000000,27.857143,...,28.065000,61.375000,49.222221,60.125000,62.750000,59.750000,65.125000,46.500000,61.375000,38.250000
20%,62.222221,63.125000,65.375000,50.222221,34.125000,48.625000,56.125000,61.333332,58.000000,44.569446,...,30.111111,64.125000,54.555557,61.666668,64.222221,60.750000,65.888885,50.000000,63.250000,54.125000
25%,63.125000,63.625000,66.000000,51.125000,36.250000,50.750000,58.138392,61.875000,59.250000,59.500000,...,34.125000,65.222221,57.875000,62.666668,65.111115,61.333332,66.444443,53.625000,64.285713,57.625000


# Check distribution drift for both datasets

In [4]:

def detect_drift(data, threshold=0.05):
  """
  Code for detecting data drift

  Args:
      data: sample NumPy array of data points.
      threshold: Proportion of data points allowed to fall outside the expected range (default: 0.05).

  Returns:
      A boolean indicating if data drift is detected.
  """
  mean = np.mean(data)
  std = np.std(data)
  expected_upper_bound = mean + 2 * std
  expected_lower_bound = mean - 2 * std

  # Calculate the cumulative sum of deviations from the mean
  deviations = np.cumsum(data - mean)

  # Identify data points exceeding the expected bounds
  outliers = np.sum(deviations > expected_upper_bound) + np.sum(deviations < expected_lower_bound)

  # Check if the proportion of outliers exceeds the threshold
  return outliers / len(data) > threshold


# Calculate drift for both datasets
drift_detected_1 = detect_drift(df1)
drift_detected_2 = detect_drift(df2)

if np.mean(drift_detected_1):
  print("Data drift detected in MetrLA data")
else:
  print("No data drift detected in MetrLA data.")
  
if np.mean(drift_detected_2):
  print("Data drift detected in PemsBay dataset")
else:
  print("No data drift detected in PemsBay data.")


/Users/paulmagos/.pyenv/versions/3.11.9/envs/Thesis/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/Users/paulmagos/.pyenv/versions/3.11.9/envs/Thesis/lib/python3.11/site-packages/numpy/core/fromnumeric.py:3462: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Data drift detected in MetrLA data
Data drift detected in PemsBay dataset


# Check drift distribution after half dataset fit with nannyml 

## MetrLa

In [5]:
METRLA_middle_index = len(df1) // 2

METRLA_first_half_df = df1.iloc[:METRLA_middle_index] # reference dataset 
METRLA_second_half_df = df1.iloc[METRLA_middle_index:]

METRLA_feature_names = df1.columns
METRLA_reference_set = METRLA_first_half_df.copy()  
METRLA_analysis_set = METRLA_second_half_df.copy() 

In [6]:
METRLA_drift_detector = nml.DataReconstructionDriftCalculator(
    column_names=METRLA_feature_names,
    chunk_size=500
).fit(reference_data=METRLA_reference_set)

# Calculate drift results
METRLA_drift_results = METRLA_drift_detector.calculate(data=METRLA_analysis_set)

# Visualize drift
METRLA_drift_results_plot = METRLA_drift_results.plot()
METRLA_drift_results_plot.show()

## PemsBay

In [9]:
PemsBay_middle_index = len(df2) // 2

PemsBay_first_half_df = df2.iloc[:PemsBay_middle_index] # reference dataset 
PemsBay_second_half_df = df2.iloc[PemsBay_middle_index:]

PemsBay_feature_names = df2.columns
PemsBay_reference_set = PemsBay_first_half_df.copy()  
PemsBay_analysis_set = PemsBay_second_half_df.copy() 

In [10]:
PemsBay_drift_detector = nml.DataReconstructionDriftCalculator(
    column_names=PemsBay_feature_names,
    chunk_size=500
).fit(reference_data=PemsBay_reference_set)

# Calculate drift results
PemsBay_drift_results = PemsBay_drift_detector.calculate(data=PemsBay_analysis_set)

# Visualize drift
PemsBay_drift_results_plot = PemsBay_drift_results.plot()
PemsBay_drift_results_plot.show()